In [4]:
import copy
from itertools import combinations
from collections import Counter


class DiffCommandsError(Exception):
    def __init__(self, words):
        self.words = words


class EachLineCommands:
    def __init__(self, line):
        operations = ['d', 'a', 'c']
        self.left_first = None
        self.left_second = None
        self.right_first = None
        self.right_second = None
        self.operation = None
        self.line = line.strip()

        if " " in line:
            raise DiffCommandsError("Cannot possibly be the commands for the diff of two files")
        if line.isspace():
            raise DiffCommandsError("Cannot possibly be the commands for the diff of two files")
        count = Counter(list(line))
        for operation in operations:
            if operation in line:
                line_number = line.strip().split(operation)
                left_number = line_number[0]
                right_number = line_number[1]
                if left_number.isspace() or right_number.isspace():
                    raise DiffCommandsError("Cannot possibly be the commands for the diff of two files")
                left_number = left_number.split(",")
                right_number = right_number.split(",")

                for i in range(len(left_number)):
                    if not left_number[i].isdigit():
                        raise DiffCommandsError("Cannot possibly be the commands for the diff of two files")
                    else:
                        if int(left_number[i]) < 0:
                            raise DiffCommandsError("Cannot possibly be the commands for the diff of two files")

                for i in range(len(right_number)):
                    if not right_number[i].isdigit():
                        raise DiffCommandsError("Cannot possibly be the commands for the diff of two files")
                    else:
                        if int(right_number[i]) < 0:
                            raise DiffCommandsError("Cannot possibly be the commands for the diff of two files")

                if count['d'] == 1 and count['a']==0 and count['c'] == 0:

                    if len(left_number) == len(right_number) == 1:

                        self.left_first = int(left_number[0]) - 1
                        self.left_second = int(left_number[0])

                        self.right_first = int(right_number[0])
                        self.right_second = self.right_first
                    elif len(left_number) == 2 and len(right_number) == 1:

                        self.left_first = int(left_number[0]) - 1
                        self.left_second = int(left_number[1])

                        self.right_first = int(right_number[0])
                        self.right_second = int(right_number[0])
                    else:
                        raise DiffCommandsError("Cannot possibly be the commands for the diff of two files")


                elif count['a'] == 1 and count['d']==0 and count['c'] == 0:

                    if len(left_number) == len(right_number) == 1:

                        self.left_first = int(left_number[0])
                        self.left_second = self.left_first

                        self.right_second = int(right_number[0])
                        self.right_first = self.right_second - 1
                    elif len(left_number) == 1 and len(right_number) == 2:

                        self.left_first = int(left_number[0])
                        self.left_second = int(left_number[0])

                        self.right_first = int(right_number[0]) - 1
                        self.right_second = int(right_number[1])
                    else:
                        raise DiffCommandsError("Cannot possibly be the commands for the diff of two files")

                elif count['c'] == 1 and count['a']==0 and count['d'] == 0:
                    if len(left_number) == 1 and len(right_number) == 1:

                        self.left_first = int(left_number[0]) - 1
                        self.left_second = int(left_number[0])

                        self.right_first = int(right_number[0]) - 1
                        self.right_second = int(right_number[0])

                    elif len(left_number) == 2 and len(right_number) == 1:

                        self.left_first = int(left_number[0]) - 1
                        self.left_second = int(left_number[1])

                        self.right_first = int(right_number[0]) - 1
                        self.right_second = int(right_number[0])
                    elif len(left_number) == 1 and len(right_number) == 2:

                        self.left_first = int(left_number[0]) - 1
                        self.left_second = int(left_number[0])

                        self.right_first = int(right_number[0]) - 1
                        self.right_second = int(right_number[1])

                    elif len(left_number) == 2 and len(right_number) == 2:

                        self.left_first = int(left_number[0]) - 1
                        self.left_second = int(left_number[1])

                        self.right_first = int(right_number[0]) - 1
                        self.right_second = int(right_number[1])
                    else:
                        raise DiffCommandsError("Cannot possibly be the commands for the diff of two files")

                self.operation = operation
                if self.left_first > self.left_second or self.right_first > self.right_second:
                    raise DiffCommandsError("Cannot possibly be the commands for the diff of two files")

                if self.left_first == 0 and self.right_first != 0:
                    raise DiffCommandsError("Cannot possibly be the commands for the diff of two files")
                break
        else:
            raise DiffCommandsError("Cannot possibly be the commands for the diff of two files ")


class DiffCommands:
    def __init__(self, filename=None):
        self.readlines = []
        self.line_command = []
        if filename is not None:
            file = open(filename, "r")
            for line in file:
                line_command = EachLineCommands(line)
                self.line_command.append(line_command)
                self.readlines.append(line.strip())
            file.close()
        if self.line_command:
            first_line = self.line_command[0]
            for next_line in self.line_command[1:]:
                if next_line.left_first - first_line.left_second != next_line.right_first - first_line.right_second:
                    raise DiffCommandsError("Cannot possibly be the commands for the diff of two files")

                if next_line.left_first == first_line.left_second or next_line.right_first == first_line.right_second:
                    raise DiffCommandsError("Cannot possibly be the commands for the diff of two files")
                first_line = next_line

    def __repr__(self):
        return "\n".join(self.readlines).strip()


class OriginalNewFiles:
    def __init__(self, filename1, filename2):
        self.left_file_lines = []
        self.right_file_lines = []
        file = open(filename1, 'r')
        for line in file:
            if not line.isspace():
                self.left_file_lines.append(line.strip())
        file.close()
        file = open(filename2, 'r')
        for line in file:
            if not line.isspace():
                self.right_file_lines.append(line.strip())
        file.close()

    def is_a_possible_diff(self, DiffCommands):
        left_file_lines = copy.deepcopy(self.left_file_lines)
        right_file_lines = copy.deepcopy(self.right_file_lines)
        if DiffCommands:
            for eachline in DiffCommands.line_command:
                if eachline.left_first > len(self.left_file_lines) or eachline.left_second > len(
                        self.left_file_lines) or eachline.right_first > len(
                    self.right_file_lines) or eachline.right_second > len(self.right_file_lines):
                    return False
                if eachline.operation == 'd':
                    for index in range(eachline.left_first, eachline.left_second):
                        left_file_lines.remove(self.left_file_lines[index])
                elif eachline.operation == 'a':
                    for index in range(eachline.right_first, eachline.right_second):
                        right_file_lines.remove(self.right_file_lines[index])
                elif eachline.operation == 'c':
                    for index in range(eachline.right_first, eachline.right_second):
                        right_file_lines.remove(self.right_file_lines[index])
                    for index in range(eachline.left_first, eachline.left_second):
                        left_file_lines.remove(self.left_file_lines[index])
            if left_file_lines == right_file_lines:
                return True
        return False

    def output_diff(self, DiffCommands):
        if self.is_a_possible_diff(DiffCommands):
            for eachline in DiffCommands.line_command:
                print(eachline.line)
                if eachline.operation == 'd':
                    for index in range(eachline.left_first, eachline.left_second):
                        print(f"< {self.left_file_lines[index]}")
                elif eachline.operation == 'a':
                    for index in range(eachline.right_first, eachline.right_second):
                        print(f"> {self.right_file_lines[index]}")
                elif eachline.operation == 'c':
                    for index in range(eachline.left_first, eachline.left_second):
                        print(f"< {self.left_file_lines[index]}")
                    print("---")
                    for index in range(eachline.right_first, eachline.right_second):
                        print(f"> {self.right_file_lines[index]}")

    def output_unmodified_from_original(self, DiffCommands):
        left_file_lines = copy.deepcopy(self.left_file_lines)
        if self.is_a_possible_diff(DiffCommands):
            for eachline in DiffCommands.line_command:
                if eachline.operation == 'd':
                    for index in range(eachline.left_first, eachline.left_second):
                        left_file_lines[index] = '...'
                elif eachline.operation == 'c':
                    for index in range(eachline.left_first, eachline.left_second):
                        left_file_lines[index] = '...'
            first_line = left_file_lines[0]
            print(first_line)
            for next_line in left_file_lines[1:]:
                if first_line == next_line:
                    if first_line =='...' and next_line =='...':
                        pass
                    else:
                        print(next_line)
                else:
                    print(next_line)
                first_line = next_line

    def output_unmodified_from_new(self, DiffCommands):
        right_file_lines = copy.deepcopy(self.right_file_lines)
        if self.is_a_possible_diff(DiffCommands):
            for eachline in DiffCommands.line_command:
                if eachline.operation == 'a':
                    for index in range(eachline.right_first, eachline.right_second):
                        right_file_lines[index] = '...'
                elif eachline.operation == 'c':
                    for index in range(eachline.right_first, eachline.right_second):
                        right_file_lines[index] = '...'
            first_line = right_file_lines[0]
            print(first_line)
            for next_line in right_file_lines[1:]:
                if first_line == next_line:
                    if first_line =='...' and next_line =='...':
                        pass
                    else:
                        print(next_line)
                else:
                    print(next_line)
                first_line = next_line
    def format_output(self, A, B):
        if A == B:
            return f"{B}"
        else:
            return f"{A},{B}"
    
    def check_all_commands_and_get_diffs(self,all_commands):
        result = []
        for commands in all_commands:
            first = commands[0]
            for second in commands[1:]:
                if second[0]<first[0] or second[1]<first[1] or second[2]<=first[2]:
                    break
                first = second
            else:
                result.append(commands)

        diffs = []
        for A in result:
            diff = DiffCommands()
            left_file_diffs = []
            right_file_diffs = []
            for i in range(len(A)):
                left_file_diffs.append(A[i][0])
                right_file_diffs.append(A[i][1])
            left_file_diffs.append(len(self.left_file_lines) + 1)
            right_file_diffs.append(len(self.right_file_lines) + 1)
            left_first = 0
            right_first = 0
            for left_second, right_second in zip(left_file_diffs, right_file_diffs):
            #for left_second in left_file_diffs:
                #for right_second in right_file_diffs:               
                #if left_second > left_first + 1 and right_second == right_first + 1:
                if left_second - left_first > 1 and right_second - right_first == 1:
                    left = self.format_output(left_first + 1, left_second - 1)
                    right = self.format_output(right_first, right_second - 1)
                    line = left + 'd' + right
                    diff.readlines.append(line)
                #elif left_second == left_first + 1 and right_second > right_first + 1:
                elif left_second - left_first == 1 and right_second - right_first > 1:
                    left = self.format_output(left_first, left_second - 1)
                    right = self.format_output(right_first + 1, right_second - 1)
                    line = left + 'a' + right
                    diff.readlines.append(line)
                #elif left_second > left_first + 1 and right_second > right_first + 1:
                elif left_second - left_first > 1 and right_second - right_first > 1:
                    left = self.format_output(left_first + 1, left_second - 1)
                    right = self.format_output(right_first + 1, right_second - 1)
                    line = left + 'c' + right
                    diff.readlines.append(line)
                left_first, right_first = left_second, right_second
            diffs.append(diff)
        diffs = sorted(diffs, key=lambda x: x.readlines)
        return diffs
        

    def get_all_diff_commands(self):
        if self.left_file_lines == self.right_file_lines:
            return [DiffCommands()]
        LCS = [[0 for _ in range(len(self.right_file_lines) + 1)] for _ in range(len(self.left_file_lines) + 1)]
        path = []
        for i in range(1,len(LCS)):
            for j in range(1,len(LCS[0])):
                left = self.left_file_lines[i-1]
                right = self.right_file_lines[j - 1]
                if left == right:
                    LCS[i][j] = LCS[i - 1][j - 1] + 1
                    path.append((i,j,LCS[i][j]))
                else:
                    LCS[i][j] = max(LCS[i][j - 1], LCS[i - 1][j])
        max_value = path[-1][-1]
        all_commands = combinations(path,max_value)
        diffs = self.check_all_commands_and_get_diffs(all_commands)
        return diffs

                    


if __name__ == "__main__":
    diff_1 = DiffCommands('diff_1.txt')
    diff_2 = DiffCommands('diff_2.txt')
    diff_3 = DiffCommands('diff_3.txt')
    pair_of_files = OriginalNewFiles('file_3_2.txt', 'file_3_1.txt')
    #print(pair_of_files.get_all_diff_commands())
    #diffs = pair_of_files.get_all_diff_commands()
    #pair_of_files.output_diff(diff_3) 
    #print(len(diffs))
    #print(diffs[0])
    #print(diff_1)
    #DiffCommands('wrong_1.txt')\
    diff_1 = DiffCommands('diff_1.txt') 
    print(diff_1)

1,2d0
3a2
5c4
7a7,8
10d10
13,16c13,15


In [34]:
DiffCommands('diff_1.txt')

0,2d0,0
3,3a1,2
4,5c3,4
7,7a6,8
9,10d10,10
12,16c12,15


In [ ]:
def output_diff(self,DiffCommands):
        if self.is_a_possible_diff(DiffCommands):
            for eachline in DiffCommands.line_command:
                print(eachline.line)
                if eachline.operation == 'd':
                    for index in range(eachline.left_first, eachline.left_second):
                        print(f"< {self.left_file_lines[index]}")                                      
                elif eachline.operation == 'a':
                    for index in range(eachline.right_first, eachline.right_second):
                        print(f"> {self.right_file_lines[index]}")                
                elif eachline.operation == 'c':
                    for index in range(eachline.right_first, eachline.right_second):
                        print(f"> {self.right_file_lines[index]}")
                    for index in range(eachline.left_first, eachline.left_second):
                        print(f"< {self.left_file_lines[index]}")
                        print("---")

In [45]:
from itertools import combinations
A=[(3, 1, 1), (4, 3, 2), (6, 5, 3), (7, 6, 4), (8, 9, 5), (9, 10, 6), (11, 11, 7), (12, 12, 8), (17, 16, 9)]
max_value = A[-1][-1]
all_commands = combinations(A,max_value)
result=[]
for commands in all_commands:
    first = commands[0]
    for second in commands[1:]:
        if second[0]<first[0] or second[1]<first[1] or second[2]<=first[2]:
            break
        first = second
    else:
        result.append(commands)
print(result)


[((3, 1, 1), (4, 3, 2), (6, 5, 3), (7, 6, 4), (8, 9, 5), (9, 10, 6), (11, 11, 7), (12, 12, 8), (17, 16, 9))]


In [47]:
A=[(3, 1, 1), (4, 3, 2), (6, 5, 3), (7, 6, 4), (8, 9, 5), (9, 10, 6), (11, 11, 7), (12, 12, 8), (17, 16, 9)]
result=[]
for i in tuple(A):
    first = tuple(A)[0]
    for second in tuple(A)[1:]:
        if second[0]<first[0] or second[1]<first[1] or second[2]<=first[2]:
            break
        first = second
    else:
        result.append(i)
print(result)
    

[(3, 1, 1), (4, 3, 2), (6, 5, 3), (7, 6, 4), (8, 9, 5), (9, 10, 6), (11, 11, 7), (12, 12, 8), (17, 16, 9)]


In [46]:
A=[(3, 1, 1), (4, 3, 2), (6, 5, 3), (7, 6, 4), (8, 9, 5), (9, 10, 6), (11, 11, 7), (12, 12, 8), (17, 16, 9)]
first = tuple(A)[0]
second = tuple(A)[1:]
second

((4, 3, 2),
 (6, 5, 3),
 (7, 6, 4),
 (8, 9, 5),
 (9, 10, 6),
 (11, 11, 7),
 (12, 12, 8),
 (17, 16, 9))

In [3]:
A=[(1,2,3),(7,8,9)]
B=[(4,5,6),(8,9,10)]
C=zip(A,B)
print(C)

In [5]:
A='hello'
B=list(A)
B

['h', 'e', 'l', 'l', 'o']